In [1]:
import os
import re
from collections import defaultdict

import textract
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from PyPDF2 import PdfReader, PdfWriter
from tqdm.autonotebook import tqdm
import pandas as pd

/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_80438/945261966.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# 1. Разбивка PDF на текстовые блоки

In [2]:
REPORTS_DIRECTORY = "reports"

RE_DERIVED = re.compile(r"\w+( -|- |-|! - )\w+")
RE_RUSSIAN_TEXT = re.compile(r"[а-яА-Яa-zA-Z0-9\-]+")

stopwords_ru = stopwords.words("russian")

In [3]:
morph = MorphAnalyzer()
cp_names = {
    "ак",
    "акб",
    "акига",
    "алмазэргиэнбанк",
    "алрос",
    "альфабанк",
    "алюминиевый",
    "амвэя",
    "ангарский",
    "анк",
    "ао",
    "архангельский",
    "ас",
    "асэ",
    "атомпроект",
    "атомредметзолото",
    "атомэнергомаш",
    "атомэнергопроект",
    "атомэнергопром",
    "афк",
    "ашан",
    "аэрофлот",
    "балтика",
    "барс",
    "бат",
    "башнефть",
    "билайн",
    "бифид",
    "бф",
    "виктория",
    "владивостокский",
    "внипиэт",
    "волга",
    "волгателек",
    "волжский",
    "восток",
    "втб",
    "выксунский",
    "вымпелком",
    "вэб",
    "газпром",
    "газпромбанк",
    "гарантинвест",
    "генерировать",
    "геннадий",
    "гидропресс",
    "гк",
    "гмк",
    "гнц",
    "головной",
    "гольцблат",
    "грэс",
    "гтлк",
    "губкин",
    "дания",
    "дивизион",
    "евразийский",
    "еврохим",
    "елена",
    "енисейский",
    "еэс",
    "желдортранс",
    "зао",
    "зарубежнефть",
    "ик",
    "илим",
    "имииафрикантовый",
    "инжиниринговый",
    "инккапитал",
    "интер",
    "интернэшнл",
    "информация",
    "иркутск",
    "казаньоргсинтез",
    "казмунайгаз",
    "калининградский",
    "камаз",
    "касперский",
    "кб",
    "кемикалс",
    "ковровский",
    "комстаротс",
    "красцветмет",
    "крафтфудс",
    "кубань",
    "кузбассэнерго",
    "ленэнерго",
    "леруа",
    "лпк",
    "лср",
    "лукойл",
    "мвидео",
    "мдмбанк",
    "мегафон",
    "мерлен",
    "металлоинвест",
    "металлургический",
    "метафракс",
    "ммк",
    "монди",
    "мон’дэлиса",
    "моррис",
    "мосводоканал",
    "московский",
    "мрск",
    "мрф",
    "мсп",
    "мтс",
    "мхк",
    "неманский",
    "нестля",
    "ниаэп",
    "нижнекамскнефтехим",
    "нииар",
    "нипигаз",
    "ниу",
    "нк",
    "нлмк",
    "новатэк",
    "новогор",
    "новосибирский",
    "нордиск",
    "норильский",
    "оао",
    "объединение",
    "огк",
    "окб",
    "окбм",
    "около",
    "омк",
    "они",
    "ооо",
    "пао",
    "пейп",
    "пепеляев",
    "петропавловск",
    "пивоваренный",
    "пивоваров",
    "пигмент",
    "пик",
    "по",
    "полиметалл",
    "приволжье",
    "прикамье",
    "рао",
    "распадский",
    "ргу",
    "реновый",
    "ржд",
    "рксхолдинг",
    "рольф",
    "росатом",
    "росводоканал",
    "роснефть",
    "россеть",
    "российский",
    "россия",
    "ростелеком",
    "ростёха",
    "росэнергоатом",
    "рудна",
    "русало",
    "русгидро",
    "руссинвест",
    "руссия",
    "русый",
    "русь",
    "рф",
    "рязанский",
    "сабмиллер",
    "санофироссия",
    "сахалин",
    "сбербанк",
    "свез",
    "севергазпром",
    "севернефтегазпром",
    "северсталь",
    "сзлк",
    "сибирь",
    "сибурхолдинг",
    "система",
    "ситибанк",
    "ситроникс",
    "совкомбанк",
    "спбаэп",
    "суал",
    "сургутнефтегаз",
    "суэк",
    "схк",
    "сыктывкарский",
    "такед",
    "таманьнефтегаз",
    "татнефть",
    "твэл",
    "тгк",
    "техснабэкспорт",
    "тимченко",
    "тмк",
    "тнк",
    "тнквр",
    "тоаз",
    "трансаэро",
    "трансгаз",
    "транснефть",
    "тюмень",
    "урал",
    "уралкалий",
    "уралсиб",
    "уральский",
    "урсабанк",
    "ухта",
    "фгк",
    "ферреро",
    "фиабанк",
    "филиал",
    "филип",
    "фк",
    "фортум",
    "фосагро",
    "фпк",
    "фск",
    "химконцентрат",
    "центринвест",
    "черкизовый",
    "чусовской",
    "шелл",
    "шереметьево",
    "щуровский",
    "эксонмобила",
    "энела",
    "энергохолдинг",
    "энерджи",
    "эсэфай",
    "южнороссийский",
    "юкос",
    "юникредитбанк",
    "юнипро",
    "яндекс",
    "ятэк",
}

In [4]:
info_df = pd.read_csv("rspp_reports.csv", index_col=0)
info_df.head()

,компания,сектор,год,тип отчета,ссылка на отчет
0,Центр Корпоративной Медицины,Здравоохранение и спорт,2022,ОУР,/download/b98f348b936bbc33387b248e52cb1f2a/
1,ДОМ.РФ,Финансы и страхование,2022,ОУР,/download/82e60c15b4c0ee6ede2e194aa31c69e1/
2,ПАО «Россети Ленэнерго»,Энергетика,2022,СО,/download/d8bc2c7e94424c65e37793a462b7807d1274...
3,"ОАО ""МРСК Урала""",Энергетика,2021,ИО,/download/af503e07dd6b861d1ed3048c36868cc9/
4,"ПАО ""Россети Сибирь""",Энергетика,2021,ИО,/download/57becde4be827f45bedf2a46f58d793a/


In [5]:
files = []
for path in os.listdir("."):
    # check if path is dir
    if os.path.isdir(path):
        files.extend([os.path.join(path, file) for file in os.listdir(path)])

In [6]:
info_df.shape, len(files)

((1429, 5), 1431)

In [7]:
info_df.tail()

,компания,сектор,год,тип отчета,ссылка на отчет
1424,Глобалтранс,"Транспорт, дорожное строительство и логистичес...",2022,NaN,Глобалтранс_1424.pdf
1425,Евраз,Металлургическая и горнодобывающая,2022,NaN,Евраз 2022_1425.pdf
1426,ММК,Металлургическая и горнодобывающая,2022,NaN,ПАО ММК 2022_1426.pdf
1427,Селигдар,Металлургическая и горнодобывающая,2022,NaN,Селигдар 2022_1427.pdf
1428,ЭЛ5,Энергетика,2022,NaN,ЭЛ5-Энерго 2022_1428.pdf


In [8]:
def parse_pdf_to_paragraph(files: list[str]):
    errors = []
    df = []
    for report_name in tqdm(files):
        if not report_name.endswith(".pdf"):
            continue
        try:
            report = textract.process(report_name)
        except Exception as e:
            errors.append((report_name, e))
            continue
        company_name = report_name[:-4].split("_")[0]
        report_index = int(report_name[:-4].split("_")[-1])
        for i, paragraph in enumerate(report.decode().split("\n\n")):
            paragraph = paragraph.replace("\n", " ").lower()
            report_page = RE_DERIVED.sub("", paragraph)
            report_page_lst = word_tokenize(report_page)
            # russian_report_page_lst = [w for w in filter(RE_RUSSIAN_TEXT.match, report_page_lst)]
            tokens = []
            # nouns = set()
            # verbs = set()
            # adj = set()
            for word_ in filter(RE_RUSSIAN_TEXT.match, report_page_lst):
                if word_ and word_ not in stopwords_ru and word_ not in cp_names:
                    word_ = word_.strip()
                    word_ = morph.parse(word_)[0]
                    normal_form = word_.normal_form
                    tokens.append(normal_form)
                    # if word_.tag.POS in NOUNS:
                    #     nouns.add(normal_form)
                    # elif word_.tag.POS in VERBS:
                    #     verbs.add(normal_form)
                    # elif word_.tag.POS in ADJ:
                    #     adj.add(normal_form)
            # if len(tokens) == 0:
            #     continue

            df.append(
                {
                    "company": company_name,
                    "year": info_df.at[report_index, "год"],
                    "sector": info_df.at[report_index, "сектор"],
                    "report_type": info_df.at[report_index, "тип отчета"],
                    "paragraph": i,
                    "original_text": paragraph,
                    "cleaned_text": " ".join(tokens),
                }
            )
    print(*errors, sep="\n")
    return df

In [9]:
df = parse_pdf_to_paragraph(files)

  0%|          | 0/1431 [00:00<?, ?it/s]

('2012/АО «ОКБМ им.И.И.Африкантова»_565.pdf', ShellError('pdftotext 2012/АО «ОКБМ им.И.И.Африкантова»_565.pdf -', 1, b'', b'Syntax Error: Document stream is empty\n'))
('2012/ПАО «Сбербанк России» _1103.pdf', ShellError('pdftotext 2012/ПАО «Сбербанк России» _1103.pdf -', 1, b'', b"Syntax Warning: May not be a PDF file (continuing anyway)\nSyntax Error (16): Illegal character '>'\nSyntax Error: Couldn't find trailer dictionary\nSyntax Error: Couldn't find trailer dictionary\nSyntax Error: Couldn't read xref table\n"))
('2002/ОАО «Рязанская ГРЭС»_1031.pdf', ShellError('pdftotext 2002/ОАО «Рязанская ГРЭС»_1031.pdf -', 1, b'', b"Syntax Warning: May not be a PDF file (continuing anyway)\nSyntax Error (26): Illegal character '>'\nSyntax Error: Couldn't find trailer dictionary\nSyntax Error: Couldn't find trailer dictionary\nSyntax Error: Couldn't read xref table\n"))
('2002/ПАО «Газпром»_380.pdf', ShellError('pdftotext 2002/ПАО «Газпром»_380.pdf -', 1, b'', b"Syntax Warning: May not be a PDF

In [10]:
new_df = pd.DataFrame(df)
new_df.head()

,company,year,sector,report_type,paragraph,original_text,cleaned_text
0,2019/ПАО «Ростелеком»,2019,Телекоммуникационная и связь,ОУР,0,отчет об устойчивом развитии,отчёт устойчивый развитие
1,2019/ПАО «Ростелеком»,2019,Телекоммуникационная и связь,ОУР,1,отчет об устойчивом развитии 2019,отчёт устойчивый развитие 2019
2,2019/ПАО «Ростелеком»,2019,Телекоммуникационная и связь,ОУР,2,хорошая цифра,хороший цифра
3,2019/ПАО «Ростелеком»,2019,Телекоммуникационная и связь,ОУР,3,ростелеком. отчет об устойчивом развитии 2019,отчёт устойчивый развитие 2019
4,2019/ПАО «Ростелеком»,2019,Телекоммуникационная и связь,ОУР,4,содержание обращение председателя совета дирек...,содержание обращение председатель совет директ...


In [11]:
new_df.to_csv("paragraphs.csv.zip")